In [2]:
import os
import shutil
import random
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Define paths
dataset_directory = './DATASETS/dataset_without_oversampling/Dataset_subset' 
output_directory = './DATASETS/Subset_Trial_Dataset'
class_folders = ['Bonafide', 'Fake']

# Define the size of each subset (percentage of the original dataset)
subset_size = 0.2  # 20% of the data will be in each subset

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through each class folder
for class_folder in class_folders:
    # Create a subdirectory for the current class in the output directory
    class_output_dir = os.path.join(output_directory, class_folder)
    os.makedirs(class_output_dir, exist_ok=True)
    
    # List all audio files in the current class folder
    audio_files = os.listdir(os.path.join(dataset_directory, class_folder))
    
    # Calculate the number of files to include in the subset
    num_files_in_subset = int(subset_size * len(audio_files))
    
    # Randomly select files for the subset
    random.shuffle(audio_files)
    subset_files = audio_files[:num_files_in_subset]
    
    # Copy the selected files to the subset directory
    for file in subset_files:
        source_path = os.path.join(dataset_directory, class_folder, file)
        destination_path = os.path.join(class_output_dir, file)
        shutil.copyfile(source_path, destination_path)
    
    print(f"Created a {subset_size*100}% subset for {class_folder}.")

Created a 20.0% subset for Bonafide.
Created a 20.0% subset for Fake.


In [1]:
# # Source and destination directories
# source_directory = './DATASETS/dataset_without_oversampling/Fake'
# destination_directory = './DATASETS/dataset_without_oversampling/Fake13100'

# os.makedirs(destination_directory, exist_ok=True)

# # List all files in the source directory
# all_files = os.listdir(source_directory)

# # Filter audio files (adjust the file extension as per your audio file types)
# audio_files = [file for file in all_files if file.endswith('.mp3') or file.endswith('.wav') or file.endswith('.ogg')]

# # Choose 13100 random audio files
# selected_files = random.sample(audio_files, 13100)

# # Copy selected files to the destination directory
# for file_name in selected_files:
#     source_path = os.path.join(source_directory, file_name)
#     destination_path = os.path.join(destination_directory, file_name)
#     shutil.copy(source_path, destination_path)

In [4]:
input_directory = './DATASETS/processed_latest/Subset'
output_directory = './DATASETS/processed_latest/Subset_Heatmaps'

In [5]:
os.makedirs(output_directory, exist_ok=True)

In [6]:
# Create 'Bonafide' and 'Fake' folders if they don't exist
for label in ['Bonafide', 'Fake']:
    label_dir = os.path.join(output_directory, label)
    os.makedirs(label_dir, exist_ok=True)

In [7]:
# Initialize progress counter
file_count = sum([len(files) for _, _, files in os.walk(input_directory)])
current_index = 0

In [8]:
# Function to extract MFCC features from audio and save as an image
def save_mfcc_as_image(audio_path, label, output_dir, index, total_files):
    y, sr = librosa.load(audio_path)

    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    
    # Plot MFCCs
    plt.figure(figsize=(15,10))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('MFCC')
    plt.tight_layout()

    # Save the plot as an image with the corresponding label and file name
    output_path = os.path.join(output_dir, label, f"{index:04d}_{os.path.splitext(os.path.basename(audio_path))[0]}.png")
    plt.savefig(output_path)
    plt.close()
    
    # Display progress
    print(f"Processed {index}/{total_files} files", end='\r')

In [ ]:
# Process each audio file in the input directory
for root, dirs, files in os.walk(input_directory):
    for file in files:
        audio_path = os.path.join(root, file)
        
        # Determine the label based on the folder name
        label = os.path.basename(root)
        
        # Extract MFCC and save as an image in the corresponding folder
        save_mfcc_as_image(audio_path, label, output_directory, current_index + 1, file_count)
        current_index += 1

print("\nProcessing completed.")

In [2]:
import os
import math
import h5py
import librosa
import numpy as np
import matplotlib.pyplot as plt

# Define constants
DATASET_PATH = './DATASETS/processed_latest/Subset'
HDF5_PATH = "LATEST_hdf5_mfcc_extracted_data.h5"
SAMPLE_RATE = 16000
SAMPLES_PER_TRACK = SAMPLE_RATE * 2  # Assuming 2 seconds per track

def save_mfcc_hdf5(dataset_path, hdf5_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=10):
    """Extracts MFCCs from the dataset and saves them into an HDF5 file along with labels and mapping.
    :param dataset_path (str): Path to dataset
    :param hdf5_path (str): Path to HDF5 file used to save MFCCs
    :param num_mfcc (int): Number of coefficients to extract
    :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
    :param hop_length (int): Sliding window for FFT. Measured in # of samples
    :param num_segments (int): Number of segments we want to divide sample tracks into
    :return:
    """
    mapping_list = []
    labels_list = []
    mfcc_list = []
    
    # Check if the HDF5 file already exists
    if os.path.exists(hdf5_path):
        raise FileExistsError(f"The file {hdf5_path} already exists. Choose a different filename or delete the existing file.")
    
    # Create empty lists to store the data
    mapping_list = []
    labels_list = []
    mfcc_list = []
    
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # Loop through all sub-folders
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # Ensure we're processing a sub-folder level
        if dirpath is not dataset_path:

            # Save label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            print("\nProcessing: {}".format(semantic_label))
            
            # Preallocate space for label and MFCC data
            labels = []
            mfccs = []
            
            # Process all audio files in sub-directory
            for j, f in enumerate(filenames):
                
                print(f"Processing file {j+1}/{len(filenames)}", end='\r')
                
                # Load audio file
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # Process all segments of audio file
                for d in range(num_segments):

                    # Calculate start and finish sample for the current segment
                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # Extract MFCC
                    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    # Store only MFCC features with the expected number of vectors
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        labels.append(i)  # Assign label index
                        mfccs.append(mfcc.tolist())
            
            # Extend the lists for this label
            # Only append data if there is data to append (same-length arrays)
            if labels and mfccs:
                mapping_list.append(semantic_label)
                labels_list.extend(labels)
                mfcc_list.extend(mfccs)

    # Create a dictionary to store the data
    data = {
        "mapping": mapping_list,
        "labels": labels_list,
        "mfcc": mfcc_list
    }
    
    # Create and open the HDF5 file
    with h5py.File(hdf5_path, 'w') as hdf5_file:
        # Create datasets for each key in the data dictionary
        for key, value in data.items():
            hdf5_file.create_dataset(key, data=value)
        
    print(f'Data has been successfully saved to {hdf5_path}.')
    return mapping_list, labels_list, mfcc_list

def save_mfcc_as_image(audio_path, label, output_dir, index, total_files):
    y, sr = librosa.load(audio_path)

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Plot MFCCs
    plt.figure(figsize=(15, 10))
    librosa.display.specshow(mfccs, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('MFCC')
    plt.tight_layout()

    # Save the plot as an image with the corresponding label and file name
    output_path = os.path.join(output_dir, label, f"{index:04d}_{os.path.splitext(os.path.basename(audio_path))[0]}.png")
    plt.savefig(output_path)
    plt.close()

    # Display progress
    print(f"Processed {index}/{total_files} files", end='\r')

def save_mfcc_heatmaps_from_hdf5(hdf5_path, output_directory):
    with h5py.File(hdf5_path, 'r') as hf:
        mapping_list = [label.decode('utf-8') for label in hf['mapping']]
        labels_list = hf['labels'][()]
        mfcc_list = hf['mfcc'][()]

    for label in set(mapping_list):
        label_dir = os.path.join(output_directory, label)
        os.makedirs(label_dir, exist_ok=True)

    file_count = len(mapping_list)
    current_index = 0

    for i, (label, mfcc) in enumerate(zip(mapping_list, mfcc_list)):
        mfcc = np.array(mfcc)

        label_dir = os.path.join(output_directory, label)

        plt.figure(figsize=(15, 10))
        librosa.display.specshow(mfcc, x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.title('MFCC')
        plt.tight_layout()

        output_path = os.path.join(label_dir, f"{i:04d}_{label}.png")
        plt.savefig(output_path)
        plt.close()

        current_index += 1
        print(f"Processed {current_index}/{file_count} files", end='\r')

# Extract MFCCs from the dataset and save to HDF5
# mapping_list, labels_list, mfcc_list = save_mfcc_hdf5(DATASET_PATH, HDF5_PATH)

# Process the extracted MFCCs and save as images
output_directory = 'D:/Sailu/Academics/BE/Major Project A/Deepfake_audio_detection/DATASETS/processed_latest/Subset/Subset_Heatmaps'
os.makedirs(output_directory, exist_ok=True)
save_mfcc_heatmaps_from_hdf5(HDF5_PATH, output_directory)

print("\nProcessing completed.")

Processed 2/2 files
Processing completed.


In [1]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

# Function to extract MFCC features and save as heatmap
def extract_mfcc(file_path, output_directory):
    y, sr = librosa.load(file_path)  # Load audio file
    num_mfcc = 40  # Number of MFCC coefficients
    n_fft = 1024  # FFT window size
    hop_length = 256  # Hop length
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T  # Transpose the MFCC matrix

    # Plotting the MFCC as a heatmap
    plt.figure(figsize=(15, 10))
    librosa.display.specshow(mfcc, x_axis='time')
    plt.colorbar()
    plt.title('MFCC Heatmap')

    # Check if the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Save the MFCC heatmaps
    output_path = os.path.join(output_directory, os.path.basename(file_path) + '_mfcc.png')
    plt.savefig(output_path)
    plt.close()
# Directories ]
bonafide_directory = 'D:/Sailu/Academics/BE/Major Project A/Deepfake_audio_detection/DATASETS/processed_latest/Subset/Bonafide'
fake_directory = 'D:/Sailu/Academics/BE/Major Project A/Deepfake_audio_detection/DATASETS/processed_latest/Subset/Fake'
output_directory_bonafide = 'D:/Sailu/Academics/BE/Major Project A/Deepfake_audio_detection/DATASETS/processed_latest/Subset_Heatmaps/Bonafide'
output_directory_fake = 'D:/Sailu/Academics/BE/Major Project A/Deepfake_audio_detection/DATASETS/processed_latest/Subset_Heatmaps/Fake'

Process bonafide files
bonafide_files = os.listdir(bonafide_directory)
for file in bonafide_files:
    if file.endswith(".wav"):  
        file_path = os.path.join(bonafide_directory, file)
        extract_mfcc(file_path, output_directory_bonafide)
        print(f"Processed bonafide file: {file}")

# Process fake files
fake_files = os.listdir(fake_directory)
for file in fake_files:
    if file.endswith(".wav"):  
        file_path = os.path.join(fake_directory, file)
        extract_mfcc(file_path, output_directory_fake)
        print(f"Processed fake file: {file}")


Processed fake file: LJ001-0001_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0001_ljspeech_waveglow.wav
Processed fake file: LJ001-0004_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0004_gen_ljspeech_melgan_large.wav
Processed fake file: LJ001-0004_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ001-0004_ljspeech_waveglow.wav
Processed fake file: LJ001-0006_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ001-0008_gen_ljspeech_melgan_large.wav
Processed fake file: LJ001-0009_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ001-0011_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0014_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ001-0015_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0015_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ001-0015_gen_ljspeech_melgan_large.wav
Processed fake file: LJ001-0017_ljspeech_waveglow.wav
Processed fake file: LJ001-0019_gen_ljspeech_melgan.wav
Processed f

Processed fake file: LJ001-0126_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ001-0127_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0127_ljspeech_waveglow.wav
Processed fake file: LJ001-0128_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0129_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ001-0129_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ001-0129_ljspeech_waveglow.wav
Processed fake file: LJ001-0133_gen_ljspeech_melgan.wav
Processed fake file: LJ001-0134_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0134_gen_ljspeech_melgan_large.wav
Processed fake file: LJ001-0135_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ001-0135_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ001-0136_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ001-0136_ljspeech_waveglow.wav
Processed fake file: LJ001-0138_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ001-0140_ljspeech_waveglow.wav
Processed fake f

Processed fake file: LJ002-0072_ljspeech_waveglow.wav
Processed fake file: LJ002-0074_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0074_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0076_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0076_ljspeech_waveglow.wav
Processed fake file: LJ002-0077_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0077_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0078_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0078_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0080_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0081_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0082_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0082_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0083_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ002-0084_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0085_gen_ljspeech_parallel_wavegan.

Processed fake file: LJ002-0195_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ002-0196_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0198_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0201_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0202_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0202_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0205_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ002-0205_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0205_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0206_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0207_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0207_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0207_ljspeech_waveglow.wav
Processed fake file: LJ002-0208_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0208_ljspeech_waveglow.wav
Processed fake file: LJ002-0211_gen_ljspeech_multi_band_m

Processed fake file: LJ002-0320_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ002-0320_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0321_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0321_ljspeech_waveglow.wav
Processed fake file: LJ002-0325_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0327_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0328_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0328_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0329_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ002-0329_gen_ljspeech_melgan.wav
Processed fake file: LJ002-0329_ljspeech_waveglow.wav
Processed fake file: LJ002-0330_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0330_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ002-0331_gen_ljspeech_melgan_large.wav
Processed fake file: LJ002-0331_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ002-0337_gen_ljspeech_multi_band_melgan.wav
Proce

...

Processed fake file: LJ014-0211_gen_ljspeech_melgan_large.wav
Processed fake file: LJ014-0212_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ014-0212_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ014-0214_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ014-0214_gen_ljspeech_melgan.wav
Processed fake file: LJ014-0215_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ014-0216_ljspeech_waveglow.wav
Processed fake file: LJ014-0217_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ014-0220_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ014-0221_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ014-0224_ljspeech_waveglow.wav
Processed fake file: LJ014-0226_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ014-0227_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ014-0227_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ014-0227_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ014-0228_gen_ljspeech_ful

Processed fake file: LJ015-0026_ljspeech_waveglow.wav
Processed fake file: LJ015-0029_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0029_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0030_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0031_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0031_gen_ljspeech_melgan.wav
Processed fake file: LJ015-0032_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0033_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ015-0037_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ015-0037_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0038_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0040_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0040_gen_ljspeech_melgan_large.wav
Processed fake file: LJ015-0043_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ015-0043_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0044_gen_l

Processed fake file: LJ015-0174_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ015-0174_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0174_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0175_gen_ljspeech_melgan.wav
Processed fake file: LJ015-0176_gen_ljspeech_melgan.wav
Processed fake file: LJ015-0176_gen_ljspeech_melgan_large.wav
Processed fake file: LJ015-0177_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ015-0179_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ015-0180_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0180_ljspeech_waveglow.wav
Processed fake file: LJ015-0182_gen_ljspeech_melgan_large.wav
Processed fake file: LJ015-0183_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ015-0185_gen_ljspeech_melgan_large.wav
Processed fake file: LJ015-0185_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0186_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ015-0186_gen_ljspeech_parallel_wave

Processed fake file: LJ015-0309_gen_ljspeech_melgan_large.wav
Processed fake file: LJ015-0309_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0310_gen_ljspeech_melgan.wav
Processed fake file: LJ015-0311_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ015-0311_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ015-0312_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ015-0314_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ016-0004_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ016-0004_ljspeech_waveglow.wav
Processed fake file: LJ016-0005_gen_ljspeech_melgan.wav
Processed fake file: LJ016-0006_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ016-0006_gen_ljspeech_melgan_large.wav
Processed fake file: LJ016-0008_gen_ljspeech_melgan.wav
Processed fake file: LJ016-0008_ljspeech_waveglow.wav
Processed fake file: LJ016-0010_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ016-0011_ljspeech_waveglow.wav
Processed fake file: LJ016-

Processed fake file: LJ016-0126_generated_ljspeech_hifiGAN.wav
Processed fake file: LJ016-0126_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ016-0127_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ016-0127_ljspeech_waveglow.wav
Processed fake file: LJ016-0128_ljspeech_waveglow.wav
Processed fake file: LJ016-0129_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ016-0129_gen_ljspeech_melgan_large.wav
Processed fake file: LJ016-0129_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ016-0130_gen_ljspeech_parallel_wavegan.wav
Processed fake file: LJ016-0132_gen_ljspeech_full_band_melgan.wav
Processed fake file: LJ016-0132_gen_ljspeech_melgan.wav
Processed fake file: LJ016-0132_ljspeech_waveglow.wav
Processed fake file: LJ016-0133_gen_ljspeech_multi_band_melgan.wav
Processed fake file: LJ016-0134_gen_ljspeech_melgan.wav
Processed fake file: LJ016-0138_gen_ljspeech_melgan.wav
Processed fake file: LJ016-0139_gen_ljspeech_melgan_large.wav
Processed fake file